In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 위성 추격-회피 게임 시작하기\n",
    "\n",
    "이 노트북은 위성 추격-회피 게임 프레임워크의 기본 사용법을 안내합니다.\n",
    "\n",
    "## 목차\n",
    "1. [환경 설정](#1.-환경-설정)\n",
    "2. [기본 환경 탐색](#2.-기본-환경-탐색)\n",
    "3. [간단한 학습](#3.-간단한-학습)\n",
    "4. [결과 분석](#4.-결과-분석)\n",
    "5. [시각화](#5.-시각화)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. 환경 설정\n",
    "\n",
    "먼저 필요한 모듈들을 임포트하고 기본 설정을 로드합니다."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 기본 라이브러리\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "from mpl_toolkits.mplot3d import Axes3D\n",
    "\n",
    "# 프로젝트 모듈\n",
    "from config.settings import get_config\n",
    "from environment.pursuit_evasion_env import PursuitEvasionEnv\n",
    "from training.trainer import create_trainer\n",
    "from analysis.evaluator import create_evaluator\n",
    "from analysis.visualization import visualize_trajectory\n",
    "\n",
    "# 설정\n",
    "plt.style.use('default')\n",
    "%matplotlib inline"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 프로젝트 설정 로드\n",
    "config = get_config(\n",
    "    experiment_name=\"notebook_tutorial\",\n",
    "    debug_mode=True  # 빠른 실행을 위해 디버그 모드\n",
    ")\n",
    "\n",
    "print(f\"실험 이름: {config.experiment_name}\")\n",
    "print(f\"GPU 사용: {config.training.use_gpu}\")\n",
    "print(f\"디버그 모드: {config.debug_mode}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. 기본 환경 탐색\n",
    "\n",
    "추격-회피 환경을 생성하고 기본 동작을 확인해봅시다."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 환경 생성\n",
    "env = PursuitEvasionEnv(config)\n",
    "\n",
    "print(f\"액션 공간: {env.action_space}\")\n",
    "print(f\"관측 공간: {env.observation_space}\")\n",
    "print(f\"최대 스텝: {env.max_steps}\")\n",
    "print(f\"포획 거리: {env.capture_distance} m\")\n",
    "print(f\"회피 거리: {env.evasion_distance} m\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 환경 리셋 및 초기 상태 확인\n",
    "obs = env.reset()\n",
    "\n",
    "print(f\"초기 관측값 형태: {obs.shape}\")\n",
    "print(f\"초기 관측값: {obs}\")\n",
    "print(f\"초기 상대 거리: {env.initial_relative_distance:.2f} m\")\n",
    "\n",
    "# 궤도 요소 출력\n",
    "evader_elements = env.initial_evader_orbital_elements\n",
    "pursuer_elements = env.initial_pursuer_orbital_elements\n",
    "\n",
    "print(f\"\\n회피자 궤도:\")\n",
    "print(f\"  반장축: {evader_elements['a']/1000:.2f} km\")\n",
    "print(f\"  이심률: {evader_elements['e']:.6f}\")\n",
    "print(f\"\\n추격자 궤도:\")\n",
    "print(f\"  반장축: {pursuer_elements['a']/1000:.2f} km\")\n",
    "print(f\"  이심률: {pursuer_elements['e']:.6f}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 몇 스텝 실행해보기\n",
    "print(\"랜덤 액션으로 몇 스텝 실행:\")\n",
    "\n",
    "for step in range(5):\n",
    "    # 랜덤 액션 생성\n",
    "    action = env.action_space.sample()\n",
    "    \n",
    "    # 스텝 실행\n",
    "    obs, reward, done, info = env.step(action)\n",
    "    \n",
    "    # 현재 상대 거리 계산\n",
    "    current_distance = np.linalg.norm(env.state[:3])\n",
    "    \n",
    "    print(f\"스텝 {step+1}: 거리 {current_distance:.2f}m, 보상 {reward:.4f}\")\n",
    "    \n",
    "    if done:\n",
    "        print(f\"에피소드 종료: {info.get('outcome', 'unknown')}\")\n",
    "        break"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. 간단한 학습\n",
    "\n",
    "SAC 알고리즘을 사용해서 간단한 학습을 실행해봅시다."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 새 환경 생성 (학습용)\n",
    "env = PursuitEvasionEnv(config)\n",
    "\n",
    "# 트레이너 생성\n",
    "trainer = create_trainer(env, config)\n",
    "trainer.setup_model()\n",
    "\n",
    "print(\"모델 설정 완료\")\n",
    "print(f\"장치: {trainer.model.device}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 짧은 학습 실행 (노트북에서는 빠른 실행을 위해)\n",
    "print(\"학습 시작...\")\n",
    "\n",
    "trainer.train(total_timesteps=2000)  # 매우 짧은 학습\n",
    "\n",
    "print(\"학습 완료!\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. 결과 분석\n",
    "\n",
    "학습된 모델을 평가해봅시다."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 모델 평가\n",
    "print(\"모델 평가 중...\")\n",
    "\n",
    "eval_results = trainer.evaluate(n_episodes=5, deterministic=True)\n",
    "\n",
    "print(f\"\\n평가 결과:\")\n",
    "print(f\"  평균 보상: {eval_results['mean_reward']:.2f}\")\n",
    "print(f\"  표준편차: {eval_results['std_reward']:.2f}\")\n",
    "print(f\"  평균 길이: {eval_results['mean_length']:.1f} 스텝\")\n",
    "print(f\"  성공률: {eval_results['success_rate']:.1%}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 상세한 평가 실행\n",
    "evaluator = create_evaluator(trainer.model, env, config)\n",
    "\n",
    "# 단일 시나리오 실행하여 궤적 데이터 수집\n",
    "scenario_result = evaluator.run_single_scenario(\n",
    "    deterministic=True,\n",
    "    save_trajectory=False\n",
    ")\n",
    "\n",
    "states, actions_e, actions_p = scenario_result['trajectory']\n",
    "metrics = scenario_result['metrics']\n",
    "info = scenario_result['info']\n",
    "\n",
    "print(f\"시나리오 결과:\")\n",
    "print(f\"  최종 거리: {metrics['final_distance_m']:.2f} m\")\n",
    "print(f\"  총 delta-v: {metrics['evader_total_delta_v_ms']:.2f} m/s\")\n",
    "print(f\"  성공 여부: {'성공' if metrics['success'] else '실패'}\")\n",
    "print(f\"  종료 조건: {info.get('termination_type', 'unknown')}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. 시각화\n",
    "\n",
    "수집된 궤적 데이터를 시각화해봅시다."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 3D 궤적 시각화\n",
    "fig = plt.figure(figsize=(12, 8))\n",
    "ax = fig.add_subplot(111, projection='3d')\n",
    "\n",
    "# 궤적 그리기\n",
    "ax.plot(states[:, 0], states[:, 1], states[:, 2], 'b-', linewidth=2, label='Trajectory')\n",
    "ax.scatter(states[0, 0], states[0, 1], states[0, 2], color='g', s=100, label='Start')\n",
    "ax.scatter(states[-1, 0], states[-1, 1], states[-1, 2], color='r', s=100, label='End')\n",
    "ax.scatter(0, 0, 0, color='k', s=150, label='Evader (Chief)')\n",
    "\n",
    "# 축 레이블\n",
    "ax.set_xlabel('x (m) - Radial')\n",
    "ax.set_ylabel('y (m) - Along-Track')\n",
    "ax.set_zlabel('z (m) - Cross-Track')\n",
    "ax.set_title('위성 추격-회피 궤적')\n",
    "ax.legend()\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 거리 변화 그래프\n",
    "distances = np.sqrt(np.sum(states[:, :3]**2, axis=1))\n",
    "\n",
    "plt.figure(figsize=(12, 6))\n",
    "plt.plot(distances, 'b-', linewidth=2)\n",
    "plt.axhline(y=env.capture_distance, color='r', linestyle='--', label='Capture Distance')\n",
    "plt.axhline(y=env.evasion_distance, color='g', linestyle='--', label='Evasion Distance')\n",
    "plt.xlabel('Time Steps')\n",
    "plt.ylabel('Distance (m)')\n",
    "plt.title('추격자-회피자 간 거리 변화')\n",
    "plt.grid(True)\n",
    "plt.legend()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 액션 분석\n",
    "fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))\n",
    "\n",
    "# 회피자 액션\n",
    "evader_magnitudes = np.linalg.norm(actions_e, axis=1)\n",
    "ax1.plot(evader_magnitudes, 'g-', linewidth=2)\n",
    "ax1.set_ylabel('Evader Action Magnitude (m/s)')\n",
    "ax1.set_title('회피자 액션 크기')\n",
    "ax1.grid(True)\n",
    "\n",
    "# 추격자 액션\n",
    "pursuer_magnitudes = np.linalg.norm(actions_p, axis=1)\n",
    "ax2.plot(pursuer_magnitudes, 'r-', linewidth=2)\n",
    "ax2.set_xlabel('Time Steps')\n",
    "ax2.set_ylabel('Pursuer Action Magnitude (m/s)')\n",
    "ax2.set_title('추격자 액션 크기')\n",
    "ax2.grid(True)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 보상 변화\n",
    "rewards = scenario_result['rewards']\n",
    "\n",
    "plt.figure(figsize=(12, 6))\n",
    "plt.plot(rewards['evader'], 'g-', label='Evader Reward', alpha=0.7)\n",
    "plt.plot(rewards['pursuer'], 'r-', label='Pursuer Reward', alpha=0.7)\n",
    "plt.axhline(y=0, color='k', linestyle=':', alpha=0.5)\n",
    "plt.xlabel('Time Steps')\n",
    "plt.ylabel('Reward')\n",
    "plt.title('Zero-Sum 게임 보상 변화')\n",
    "plt.legend()\n",
    "plt.grid(True)\n",
    "plt.show()\n",
    "\n",
    "# Zero-Sum 검증\n",
    "reward_sum = np.array(rewards['evader']) + np.array(rewards['pursuer'])\n",
    "print(f\"보상 합계 평균: {np.mean(reward_sum):.6f} (0에 가까워야 함)\")\n",
    "print(f\"보상 합계 표준편차: {np.std(reward_sum):.6f}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 결론\n",
    "\n",
    "이 노트북에서는 위성 추격-회피 게임 프레임워크의 기본 사용법을 살펴보았습니다:\n",
    "\n",
    "1. **환경 설정**: 설정 로드 및 환경 생성\n",
    "2. **환경 탐색**: 액션/관측 공간 확인 및 기본 동작\n",
    "3. **모델 학습**: SAC 알고리즘을 사용한 강화학습\n",
    "4. **결과 분석**: 학습된 모델의 성능 평가\n",
    "5. **시각화**: 궤적, 거리, 액션, 보상 분석\n",
    "\n",
    "### 다음 단계\n",
    "\n",
    "- `examples/` 디렉토리의 다른 예시들 확인\n",
    "- Nash Equilibrium 학습 시도\n",
    "- 사용자 정의 설정으로 실험\n",
    "- 더 긴 학습으로 성능 개선\n",
    "\n",
    "### 추가 리소스\n",
    "\n",
    "- [프로젝트 문서](../README.md)\n",
    "- [API 참조](../docs/)\n",
    "- [예시 스크립트](../examples/)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 환경 정리\n",
    "env.close()\n",
    "print(\"노트북 완료!\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.9.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}